This is a surprisingly effective algorithm for sentiment analysis. It is surprising not because it works particularly well, but because it works at all.

I originally saw this video by Sentdex (https://www.youtube.com/watch?v=jkdWzvMOPuo) and wanted to try it on some different data sets and give it a go for myself.

The basic idea is that you can do rudimentary sentiment analysis by using k-nearest-neighbours on the length of the input after being compressed. I do not intend to question why this works, I was just interested in seeing that it worked on a variety of different data sets.

With this in mind, I googled 'datasets for sentiment analysis' and found these data sets: https://www.kaggle.com/datasets/marklvl/sentiment-labelled-sentences-data-set?resource=download https://www.kaggle.com/datasets/cosmos98/twitter-and-reddit-sentimental-analysis-dataset

In [422]:
import zipfile
with zipfile.ZipFile("sentiment analysis.zip", 'r') as zip_ref:
    zip_ref.extractall("sentiment analysis")

To start, I load the data into a dataframe (the csv versions of the files are a little weird so I did it with the text versions of the files). I then shuffle the data about with df.sample(frac = 1) which shuffles the data and returns the whole data frame.

I reset the index, then split the dataframe into training and test sets. I have found that somewhere between 60 and 80 percent of the data tends to give good results.

In [423]:
import pandas as pd
import math

file_path = "sentiment analysis/sentiment labelled sentences/amazon_cells_labelled.txt"
df = pd.read_csv(file_path, sep="\t", lineterminator="\n", header=None, names=["Sentence", "Sentiment"])

df = df.sample(frac = 1).reset_index()
train_split = 0.75

train = df.loc[0:math.floor((len(df)*train_split))-1]
test = df.loc[math.ceil(len(df)*train_split):len(df)]

This function allows us to compare the lengths of two compressed strings. We have to normalise the lengths to get the 'distance' between the two strings.

NCD means normalised compression differences, Sentdex covers this in more detail.

In [424]:
import gzip

def ncd(str1: str, str2: str) -> float:

  str1_compressed = len(gzip.compress(bytes(str1, 'utf-8')))
  str2_compressed = len(gzip.compress(bytes(str2, 'utf-8')))

  str1str2 = len(gzip.compress(bytes(" ".join([str1,str2]), 'utf-8')))

  return (str1str2 - min(str1_compressed, str2_compressed)) / max(str1_compressed, str2_compressed)

This line of code calculates the NCD to every point in the training data set, for every point in the test data set. This line is the slowest part of the code, as it does all the heavy lifting.

There is probably a good optimisation for this, as at the moment I think it is O(N*M) where N is the number of samples in the test data set and M is the number of samples in the training data set.

Seeing as we are only interested in the k closest points, you could probably do a tree map optimisation and that would definitely speed it up a lot for larger data sets.

In [425]:
ncds = [[ncd(j, i) for j in train['Sentence']] for i in test['Sentence']]

This is my (very) rudimentary implementation of k nearest neighbouts. You can pass in the pre-calculated ncds, or they can be recalculated with every use of the function.

We loop through every test point in ncds, collecting the sentiment value for its k nearest neighbours, before taking the average of all the sentiment values for that test point and then rounding to the nearest whole number. This represents the prediction of the algorithm. This is appended to a predictions array, which is ultimately returned after predicting for all the test points.

In [426]:
from numpy import average
import copy

def kNN(k: int, train: pd.DataFrame, test: pd.DataFrame, ncds: list() = None) -> list():

  predictions = []
  temp_ncds = copy.deepcopy(ncds)
  if(temp_ncds == None):
    temp_ncds = [[ncd(j, i) for j in train['Sentence']] for i in test['Sentence']]

  for testSample in temp_ncds:

    sentiments = []
    for _ in range(k):

      nearestNeighbourIndex = testSample.index(min(testSample))
      sentiments.append(train['Sentiment'][nearestNeighbourIndex])
      testSample[nearestNeighbourIndex] = float("inf")

    predictions.append(round(average(sentiments)))

  return predictions

This block tests the algorithm for different values of k from 1 to 15, printing out the accuracy of the algorithm at the end. Randomly assigning sentiment would result in a score of around 50% for this dataset. In my tests, this algorithm consistently score above 60%, with k=3 and k=5 frequently nearing 70%.

In [427]:
for k in range(1, 16, 2):
  preds = kNN(k, train, test, ncds)
  score = sum([1 for i in range(len(preds)) if preds[i] == test['Sentiment'][i+math.ceil(len(df)*train_split)]])
  accuracy = score / len(test)
  print("Accuracy at k =", k, ":", round(accuracy*100, 2), "%")

Accuracy at k = 1 : 65.2 %
Accuracy at k = 3 : 68.4 %
Accuracy at k = 5 : 66.8 %
Accuracy at k = 7 : 66.8 %
Accuracy at k = 9 : 64.0 %
Accuracy at k = 11 : 64.0 %
Accuracy at k = 13 : 64.4 %
Accuracy at k = 15 : 62.4 %


The rest of this notebook just applies the technique to different data sets. This dataset comes from imdb.

In [428]:
file_path = "sentiment analysis/sentiment labelled sentences/imdb_labelled.txt"
df = pd.read_csv(file_path, sep="\t", lineterminator="\n", header=None, names=["Sentence", "Sentiment"])

df = df.sample(frac = 1).reset_index()

train = df.loc[0:math.floor((len(df)*train_split))-1]
test = df.loc[math.ceil(len(df)*train_split):len(df)]

In [429]:
ncds = [[ncd(j, i) for j in train['Sentence']] for i in test['Sentence']]

Randomly assigning sentiment would result in a score of around 50% for this dataset. In my tests, this algorithm consistently score above 60%, with k=3 and k=5 frequently getting to 66%-67%.

In [430]:
for k in range(1, 16, 2):
  preds = kNN(k, train, test, ncds)
  score = sum([1 for i in range(len(preds)) if preds[i] == test['Sentiment'][i+math.ceil(len(df)*train_split)]])
  accuracy = score / len(test)
  print("Accuracy at k =", k, ":", round(accuracy*100, 2), "%")

Accuracy at k = 1 : 60.96 %
Accuracy at k = 3 : 67.38 %
Accuracy at k = 5 : 63.64 %
Accuracy at k = 7 : 66.31 %
Accuracy at k = 9 : 64.17 %
Accuracy at k = 11 : 66.31 %
Accuracy at k = 13 : 68.98 %
Accuracy at k = 15 : 63.64 %


In [431]:
file_path = "sentiment analysis/sentiment labelled sentences/yelp_labelled.txt"
df = pd.read_csv(file_path, sep="\t", lineterminator="\n", header=None, names=["Sentence", "Sentiment"])

df = df.sample(frac = 1).reset_index()

train = df.loc[0:math.floor((len(df)*train_split))-1]
test = df.loc[math.ceil(len(df)*train_split):len(df)]

In [432]:
ncds = [[ncd(j, i) for j in train['Sentence']] for i in test['Sentence']]

Randomly assigning sentiment would result in a score of around 50% for this dataset. In my tests, this algorithm consistently score above 60%, with k=3 and k=5 frequently getting to 62%-63%.

Interesting to note that the algorithm is less effective for this dataset from yelp, than it was for either amazon reviews or imdb.

In [433]:
for k in range(1, 16, 2):
  preds = kNN(k, train, test, ncds)
  score = sum([1 for i in range(len(preds)) if preds[i] == test['Sentiment'][i+math.ceil(len(df)*train_split)]])
  accuracy = score / len(test)
  print("Accuracy at k =", k, ":", round(accuracy*100, 2), "%")

Accuracy at k = 1 : 66.0 %
Accuracy at k = 3 : 62.0 %
Accuracy at k = 5 : 63.2 %
Accuracy at k = 7 : 60.8 %
Accuracy at k = 9 : 61.6 %
Accuracy at k = 11 : 62.8 %
Accuracy at k = 13 : 63.6 %
Accuracy at k = 15 : 63.2 %


In [434]:
with zipfile.ZipFile("twitter+reddit.zip", 'r') as zip_ref:
    zip_ref.extractall("twitter+reddit")

These next two datasets are slightly different than the first three, they have positive, negative and neutral sentiment labels attached. They are also several orders of magnitude larger. This means when I shuffle the data, I'm actually only using frac = 0.05 so I get a random 5% of the data to work with. This leads to more variability.

In [483]:
file_path = "twitter+reddit/Reddit_Data.csv"
df = pd.read_csv(file_path, sep=",", lineterminator="\n", names=["Sentence", "Sentiment"])

df = df.sample(frac = 0.05).reset_index()
train_split = 0.75

df['Sentence'] = df['Sentence'].astype('string')
df['Sentiment'] = df['Sentiment'].astype('int')

train = df.loc[0:math.floor((len(df)*train_split))-1]
test = df.loc[math.ceil(len(df)*train_split):len(df)]

In [484]:
ncds = [[ncd(str(j), str(i)) for j in train['Sentence']] for i in test['Sentence']]

Randomly assigning sentiment would result in a score of around 33.34% for this dataset. In my tests, this algorithm consistently score above 40%, with k = 5 even getting as high as 48%.

In [485]:
for k in range(1, 16, 2):
  preds = kNN(k, train, test, ncds)
  score = sum([1 for i in range(len(preds)) if preds[i] == test['Sentiment'][i+math.ceil(len(df)*train_split)]])
  accuracy = score / len(test)
  print("Accuracy at k =", k, ":", round(accuracy*100, 2), "%")

Accuracy at k = 1 : 47.53 %
Accuracy at k = 3 : 46.24 %
Accuracy at k = 5 : 48.39 %
Accuracy at k = 7 : 47.53 %
Accuracy at k = 9 : 45.81 %
Accuracy at k = 11 : 45.16 %
Accuracy at k = 13 : 45.16 %
Accuracy at k = 15 : 44.52 %


In [512]:
file_path = "twitter+reddit/Twitter_Data.csv"
df = pd.read_csv(file_path, sep=",", lineterminator="\n", names=["Sentence", "Sentiment"])

df = df.sample(frac = 0.01).reset_index()
train_split = 0.75

df['Sentence'] = df['Sentence'].astype('string')
df['Sentiment'] = df['Sentiment'].astype('int')

train = df.loc[0:math.floor((len(df)*train_split))-1]
test = df.loc[math.ceil(len(df)*train_split):len(df)]

Even on 1% of the data, this one took more than 5 mins for me

In [513]:
ncds = [[ncd(j, i) for j in train['Sentence']] for i in test['Sentence']]

Randomly assigning sentiment would result in a score of around 33.34% for this dataset. In my tests, this algorithm consistently score around 40% across all the values of k.

In [515]:
for k in range(1, 16, 2):
  preds = kNN(k, train, test, ncds)
  score = sum([1 for i in range(len(preds)) if preds[i] == test['Sentiment'][i+math.ceil(len(df)*train_split)]])
  accuracy = score / len(test)
  print("Accuracy at k =", k, ":", round(accuracy*100, 2), "%")

Accuracy at k = 1 : 48.89 %
Accuracy at k = 3 : 42.75 %
Accuracy at k = 5 : 39.8 %
Accuracy at k = 7 : 40.79 %
Accuracy at k = 9 : 41.03 %
Accuracy at k = 11 : 39.56 %
Accuracy at k = 13 : 37.1 %
Accuracy at k = 15 : 38.08 %
